# Bézier Curves

In [ ]:
%matplotlib inline
from scipy import *
from scipy.interpolate import BPoly
from numpy import *
from bqplot import *
from pylab import *
import bqplot.pyplot as plt
matplotlib.rc('text.latex', preamble=r'\usepackage{amsmath}')
matplotlib.rcParams['text.latex.preamble']=r"\usepackage{amsmath}"

In [ ]:
%%html
<style>
div.input {
    display:none;
}
</style>

In [ ]:
# Lump data
lump = ([[55,60],[40,67],[15,53],[1,60]], 
        [[1,60], [57,96],[37,86],[64,110]],
        [[64,110], [85,120], [83,25],[68,25]],
        [[68,25],[50,35],[50,106],[79,95]],
        [[79,95], [105,60], [225,80],[290,107]],
        [[290,107], [295,104], [285,93], [268, 90]],
        [[270,90], [275,0], [265,10],[255, 54]],
        [[255,54], [215,70], [205, 56], [106, 56]],
        [[106,56],[101,44],[98,32], [95,20]])


# extract x and y coefficients
cx = array([[item[0] for item in clist] for clist in lump])
cy = array([[item[1] for item in clist] for clist in lump])
s = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# build parametric polynomials x(t), y(t)
x = BPoly(cx.T, s)
y = BPoly(cy.T, s)
t = linspace(0,9,200)
plot(x(t), y(t))
ax = gca()

# force equal aspect ratio, remove spines and ticks
ax.set_aspect('equal', adjustable='box')
ax.axes.xaxis.set_ticks([])
ax.axes.yaxis.set_ticks([])
for item in ax.spines.values():
    item.set_visible(False)

Bézier curves are ubiquitous.  They are easy to implement, and can be used to model curves on manufactured goods:  cars, airplanes, shoes...or even sketches, like the dachshund above. All computer graphics routines use Bezier curves, as do all PostScript and Truetype fonts.

But what are they, and where did they come from? 

Pierre Bézier was an engineer working for the French car company Renault.  In the 1950's, car companies like Renault and Citroën used physical models, often in clay, for development purposes.  When going from development to production, these models were scaled up (*tediously*, using French curves and flexible spline rulers), a 'master model' was made, and then parts designed based on it -- often relying on the skill and ingenuity of workers to best approximate the shape of the model.  This method left significant room for error and inconsistencey.  There was a need for a common 'curve language' to systematize the process.

Bézier implemented a clever method that he learned from Prof. Onésime Durand, a brilliant mathematician that had been one of his instructors.  Prof. Durand's insight was that the shape of a curve can be characterized by a dynamic polygonal line (shown dashed below), and that by manipulating this containing broken line one could alter the curve to the desired shape.

In [ ]:
# reproduce Figrue 3.1 using matplotlib

points = [(-1.0, 1), (-0.5, 2.5), (0.2, 3), (0.35, 2.3)]
xPts = [item[0] for item in points]
yPts = [item[1] for item in points]
plot(xPts, yPts, 'bo')
#arrow(0, 0, xPts[0], yPts[0], head_width = 0.05, head_length = 0.1, lw = 0.5)
# plot arrows from origin to each point
for point in points:
    annotate("", xy=point, xytext = (0,0), arrowprops={'arrowstyle': '-|>', 'shrinkB': 5}, va='center')

# add dashed lines connecting points
for i in range(len(points)-1):
    xdash = (xPts[i], xPts[i+1])
    ydash = (yPts[i], yPts[i+1])
    plot(xdash, ydash, 'k--')
    
# label actual points
text(xPts[0] - 0.1, yPts[0] - 0.1, r"$P_0$", fontsize = 'large')
text(xPts[1] - 0.05, yPts[1] + 0.14, r"$P_1$", fontsize = 'large')
text(xPts[2] + 0.03, yPts[2] + 0.03, r"$P_2$", fontsize = 'large')
text(xPts[3] + 0.03, yPts[3] - 0.0, r"$P_3$", fontsize = 'large')

# label arrows
text(xPts[0]/2 - 0.2, yPts[0]/2 - 0.15, r"$\bf{p_0} = \bf{a_0}$", fontsize = 'large')
text(xPts[1]/2 - 0.12, yPts[1]/2 + 0.14, r"$\bf{p_1}$", fontsize = 'large')
text(xPts[2]/2 - 0.08, yPts[2]/2 + 0.03, r"$\bf{p_2}$", fontsize = 'large')
text(xPts[3]/2 + 0.03, yPts[3]/2 - 0.0, r"$\bf{p_3}$", fontsize = 'large')

# Label dashed segments
cX1 = (xPts[0]+xPts[1])/2
cX2 = (xPts[2]+xPts[1])/2
cX3 = (xPts[2]+xPts[3])/2
cY1 = (yPts[0]+yPts[1])/2
cY2 = (yPts[2]+yPts[1])/2
cY3 = (yPts[2]+yPts[3])/2

text(cX1 - 0.1, cY1, r"$\bf{a_1}$", fontsize = 'large')
text(cX2 - 0.08, cY2+0.06, r"$\bf{a_2}$", fontsize = 'large')
text(cX3 + 0.05, cY3, r"$\bf{a_3}$", fontsize = 'large')
# create curve
s = [0,1]
x = BPoly([[c] for c in xPts], s)
y = BPoly([[c] for c in yPts], s)
t = linspace(0, 1, 200)
bezier = plot(x(t), y(t))

ax = gca()
ax.axes.xaxis.set_ticks([])
ax.axes.yaxis.set_ticks([])
for item in ax.spines.values():
    item.set_visible(False)
ylim([0, 3.0]);

Consider the polygonal line defined in the above figure with four vertices {$P_0$, $P_1$, $P_2$, and $P_3$} with position vectors $\{\bf{p_i}\}_{i=0}^3$.  Label the vectors pointing along the segments $\bf{a}$, e.g. $\bf{a_1} = \bf{p_1} - \bf{p_0}$, etc.  (These $\boldsymbol{a_i}$ vectors are assumed to be linearly independent).  In general, this could be done with three vertices, or six vertices; for simplicity, let's just stick with four for now.  (As it turns out, this is pretty much what is always used, as it is both simple and flexible enough for most applications).  Then the blue curve can be represented as the parametric equation 

$$\boldsymbol{r}(t) = \boldsymbol{a}_0 + f_1(t) \boldsymbol{a}_1  + f_2(t) \boldsymbol{a}_2 + f_3(t) \boldsymbol{a}_3$$

with $0 \le t \le 1$.  These coefficient functions can be identified by imposing conditions on the curve.

First, require the starting point of the curve to be $P_0$ and the ending point to be $P_3$.  Thus
$\boldsymbol{r}(0) = \boldsymbol{a}_0 = \boldsymbol{p}_0$ and $\boldsymbol{r}(1) = \boldsymbol{p}_3$.  So

$$f_1(0) = f_2(0) = f_3(0) = 0 \text{ and } f_1(1) = f_2(1) = f_3(1) = 1$$

Then, require that the tangent to the curve at its beginning ($t=0$) be parallel to $\boldsymbol{a}_1$, and the tangent at the end of the curve be parallel to $\boldsymbol{a}_3$.  Thus:

$$\boldsymbol{r}'(0) = f_1'(0) \boldsymbol{a}_1  + f_2'(0) \boldsymbol{a}_2 + f_3'(0) \boldsymbol{a}_3 = k\boldsymbol{a}_1$$
and
$$\boldsymbol{r}'(1) = f_1'(1) \boldsymbol{a}_1  + f_2'(1) \boldsymbol{a}_2 + f_3'(1) \boldsymbol{a}_3 = k\boldsymbol{a}_3$$
which implies
$$\Rightarrow f_2'(0) = f_3'(0) = f_3'(1) = f_2'(1) = 0$$

Finally, allow the curvature at the two endpoints to only be influenced by the two nearest polygonal segments.  For example, $\boldsymbol{r}''(0)$ is determined only by {$\boldsymbol{a}_1$, and $\boldsymbol{a}_2$}.  Thus

$$\boldsymbol{r}''(0) = f_1''(0) \boldsymbol{a}_1  + f_2''(0) \boldsymbol{a}_2 + f_3''(0) \boldsymbol{a}_3 = k\boldsymbol{a}_1 + l \boldsymbol{a}_2$$
and
$$\boldsymbol{r}''(1) =f_1''(1) \boldsymbol{a}_1  + f_2''(1) \boldsymbol{a}_2 + f_3''(1) \boldsymbol{a}_3 = k\boldsymbol{a}_2+ l \boldsymbol{a}_3$$

$$\Rightarrow f_3''(0) = f_1''(1)  = 0$$

There are now twelve independent constraints on the functions chosen for the coefficients.  If cubic polynomials are used (which they are), each polynomial has four coefficients, and $4 \times 3 = 12$, so the constraints completely determine the polynomials.  It remains an exercise in elementary algebra to complete the determination.  This Prof. Durand had some clever ideas, no?

Oh, one point of clarification before we get to the algebra:  In an interview conducted in 1990, Bézier admitted that there was no Prof. Onésime Durand; he was complete fiction.  Bézier felt that if he presented this idea as his own (which it was), that others at Renault would not have confidence in it (one probably *should* suspect math methods developed by engineers...).  So he made up a fictional mathematician to whom to give credit so that the method would be more respected!

Another curious historical note:  As you can see if you look up Bézier Curves on Wikipedia, Bézier curves were actually first developed not by Bézier (or his Prof. Durand), but by Paul de Casteljau in 1959 (in an alternate and more computationally stable form).  de Casteljau worked, oddly enough, for rival car manufacturer Citroën (seriously, what are the odds of these useful curves being developed almost simultaneously by two people working at rival *French* automobile manufacturers??).  Unlike Renault, however, Citroën was highly proprietary about de Casteljau's work, and it remained secret until 1975, long after Bézier curves were widely publicized in the 1960's.

### The Algebra

Let $f_i(t) = \alpha_i t^3 + \beta_i t^2 + \gamma_i t + \delta_i$ for $i = 1, 2, 3$

The restrictions on values of the functions: 
$$f_1(0) = f_2(0) = f_3(0) = 0 \Rightarrow \delta_1 = \delta_2 = \delta_3 = 0$$

$$f_1(1) = f_2(1) = f_3(1) = 1 \Rightarrow \begin{cases} \alpha_1 + \beta_1 + \gamma_1 = 1 \\ \alpha_1 + \beta_1 + \gamma_1 = 1 \\ \alpha_1 + \beta_1 + \gamma_1 = 1 \end{cases}$$

Now use restrictions on the derivative, $f_i'(t) = 3\alpha_i t^2 + 2 \beta_i t + \gamma_i$:

$$f_2'(0) =  f_3'(0) = 0 \Rightarrow \gamma_2 = \gamma_3 = 0$$

$$f_1'(1) = f_2'(1) = 0 \Rightarrow \begin{cases} 3 \alpha_1 + 2 \beta_1 + \gamma_1 = 0 \\ 3 \alpha_2 + 2 \beta_2 = 0 \end{cases}$$

The restrictions on the second derivative:

$$f_3''(1) = f_1''(0) \Rightarrow \begin{cases} \beta_3 = 0 \\ 6 \alpha_1 + 2 \beta_1 = 0 \end{cases}$$

Putting it all together gives:

$$ \begin{pmatrix} \alpha_1 & \alpha_2 & \alpha_3 \\ \beta_1 & \beta_2 & \beta_3 \\ \gamma_1 & \gamma_2 & \gamma_3 
\\ \delta_1 & \delta_2 & \delta_3 \end{pmatrix} =\begin{pmatrix} 1 & -2 & 1 \\ -3 & 3 & 0 \\ 3 & 0 & 0 \\
0 & 0 & 0 \end{pmatrix}$$

So that the vector parametric equation for the curve is:

$$ \boldsymbol{r}(t) = \boldsymbol{a}_0 + (t^3 -3t^2 +3t)\boldsymbol{a}_1 + (-2t^3 + 3t^2) \boldsymbol{a}_2 + t^3 \boldsymbol{a}_3, \qquad 0 \le t \le 1$$

Bézier was allowed by Renault to publish his (or, rather, Prof. Durand's) work, which includes not only the results shown here, but the equivalent general results for using $n$ segments to describe the curve (which I am omitting both in the interest of time and because, while interesting, it is seldom used in practice).

A Prof. Robin Forrest at Cambridge University realized that a more convenient representation of this result could be had by replacing the segment vectors $\boldsymbol{a}_i$ with their definitions in terms of the actual point vectors $\boldsymbol{p}_i$, and then collecting in terms of the $\boldsymbol{p}_i$.    The resulting equation is then:

$$ \boldsymbol{r}(t) = (1-t)^3 \boldsymbol{p}_0 + 3t(1-t)^2\boldsymbol{p}_1 + 3t^2(1-t) \boldsymbol{p}_2 + t^3 \boldsymbol{p}_3, \qquad 0 \le t \le 1$$

(for the 3-segment, cubic polynomial coefficient version that we have been considering).  These polynomial coefficients are referred to as *blending functions*, as they determine how much weight each of the control points has on the resulting curve.  But they also provide us with another connection:  they are the terms in the binomial expansion:

$$ 1 = 1^n = (t + (1-t))^n = \sum_{i=0}^n \binom{n}{i} t^i (1-t)^{n-i} = \sum_{i=0}^n B_i^n(t),$$

where the $B_i^n(t)$, known as the *Bernstein polynomials*, are named after Russian mathematician Sergei Natanovich Bernstein (1880-1968), who used them as an alternative basis set to the usual $\{1, t, t^2, t^3, ...\}$ basis.  More on this later.

## Explore Bézier Curves

The plot below calculates the Bézier Curve using the form above in terms of the control points.  The control points are shown as blue dots; you can select them and move them interactively with a mouse to see how it affects the resulting curve.

In [ ]:
# interactive Bezier curve with control points that can be moved with mouse
# use some real numbers in control points to force real number coordinates

axes_options = {'x': {'label': 'x'}, 'y': {'label': 'y'}}
points = [(2.0,1), (4,7), (10.2,9), (12,6.7)]
xPts = [item[0] for item in points]
yPts = [item[1] for item in points]
fig = plt.figure(title = "Bézier Curve", animation_duration = 1000)
control = plt.scatter(xPts, yPts, colors=['blue'], axes_options = axes_options,
           interactions={'click': 'select'},
           enable_move = True,
           selected_style={'opacity':1.0, 'fill':'Red', 'stroke': 'Black'})
segment = [0,1]
x = BPoly([[c] for c in xPts], segment)
y = BPoly([[c] for c in yPts], segment)
curve_t = linspace(0, 1, 200)
bezier = plt.plot(x(curve_t), y(curve_t))
plt.ylim(0, 10)

# callback function
def update_line(change):
    x = BPoly([[c] for c in control.x], segment)
    y = BPoly([[c] for c in control.y], segment)
    with bezier.hold_sync(): 
        bezier.x = x(curve_t)
        bezier.y = y(curve_t)
    plt.ylim(0, 10)
    plt.xlim(0, 12)
        
control.observe(update_line, names = ['x'])
control.observe(update_line, names = ['y'])

fig

### The Weierstrass Approximation Theorem

As it turns out, the Bernstein polynomials connect us back to Weierstrass through the above-named theorem, which states (roughly, in words):  "A polynomial of sufficiently high order can uniformly approximate any continuous function over any finite interval."

Bernstein used his eponymous polynomials to provide the first constructive proof of the Weierstrass Theorem.  This is a curious sort of reversal from where we started; instead of using the Bernstein polynomials to approximate a curve by changing a function (in our case, the polygonal line control function), this uses the curve to approximate the function itself.

Here's how it works (shown without proof, but with a hopefully instructive illustration at the end):

For a continuous function $f(t)$, construct the *Bernstein polynomial function,* $B_n(f,t)$, in terms of the Bernstein polynomials:

$$ B_n(f,t) = \sum_{i=0}^n f \left( \frac{i}{n} \right) B_i^n(t)$$

That's it.  Bernstein proved that $\lim_{n \rightarrow \infty} B_n(f,t) = f(t)$ over the interval $t \in [0,1]$.  Since a simple argument transformation of the form 
$$ t \rightarrow \frac{t-a}{b-a}$$
will transform the interval $[a,b]$ into $[0,1]$, the result is achieved.



In [ ]:
# reproduce figure 3.5

axes_options = {'x': {'label': 'x'}, 'y': {'label': 'y'}}
points = [(0,0), (0.2,0.6), (0.6,0.8), (0.9,0.7), (1,0)]
xPts = [item[0] for item in points]
yPts = [item[1] for item in points]
plot(xPts, yPts, 'b-', lw = 2.5, label = r'$f(x)$')
grid()

# somewhat clumsy functions for defining piecewise f(x)
def find_int(x):
    xPts.append(x)
    idx = sorted(xPts).index(x)
    xPts.remove(x)
    return idx

def lin_int(x):
    if x in xPts:
        return yPts[xPts.index(x)]
    idx = find_int(x)
    return yPts[idx-1] + (yPts[idx] - yPts[idx-1])/(xPts[idx]-xPts[idx-1]) * (x-xPts[idx-1])

# Bernstein polynomial function:
def Bnf( n = 2):
    x = linspace(0, 1, n+1)
    s = [0, 1]
    c = [[lin_int(item)] for item in x]
    return BPoly(c, s)

t = linspace(0, 1, 100)
for n in [2,4,10,20,40]:
    f = Bnf(n)
    plot(t, f(t), label = r'$B_{%d}(f,x)$' % n)

title("Approximating a continuous function");
xlabel("x")
ylabel('y');
#legend();

Above is an illustration of this using as the target function $f(x)$ the polygonal line joining the points (0, 0), (0.2, 0.6), (0.6, 0.8), (0.9, 0.7), and (1, 0).  Shown is the original function (thick blue line) and the Bernstein polynomial function approximations $B_2(f,x)$, $B_4(f,x)$, $B_{10}(f,x)$, $B_{20}(f,x)$, and $B_{40}(f,x)$.    While these polynomials do converge to the function, the rate of convergence is glacial; too slow to be of much practical use.

### Applications

#### The Dachshund, Lump

The opening illustration uses Bézier curves to (roughly) reproduce a Picasso scribble of the dachshund "Lump".  You can read more about this dog, who was owned by David Douglas Duncan, but who apparently lived with Picasso, on the Wikipedia page [Lump (dog)](https://en.wikipedia.org/wiki/Lump_(dog)).

This is (an image of) the original sketch:

<img src="Picasso_lump.jpg" alt="Drawing" style="width: 300px;"/>

And here is the Bézier version again, this time showing the break points between the nine Bézier curves used:

In [ ]:
# extract x and y coefficients
cx = array([[item[0] for item in clist] for clist in lump])
cy = array([[item[1] for item in clist] for clist in lump])
cxPts = [clist[0][0]  for clist in lump]
cxPts.append(lump[-1][-1][0])
cyPts = [clist[0][1]  for clist in lump]
cyPts.append(lump[-1][-1][1])
s = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# build parametric polynomials x(t), y(t)
xDog = BPoly(cx.T, s)
yDog = BPoly(cy.T, s)
t = linspace(0,9,200)
plot(xDog(t), yDog(t))
plot(cxPts, cyPts, 'bo')
ax = gca()

# force equal aspect ratio, remove spines and ticks
ax.set_aspect('equal', adjustable='box')
ax.axes.xaxis.set_ticks([])
ax.axes.yaxis.set_ticks([])
for item in ax.spines.values():
    item.set_visible(False)

### Fonts

Many modern computer fonts describe each letter in terms of sets of Bézier curves.  As an example, the PostScript-based Times Roman font capital letter "O" uses 6 Bézier curves for its outer curve and 8 more for its inner curve.  By using Bézier curves, it is quick and easy to reproduce the same shape in different sizes.  (Excuse me for not typing in the four 2-dimensional control points for each of these 14 curves, each of which has up to 9 significant figures -- I've probably already spent enough time on this notebook).

#### Implementation notes:

As in previous notebooks, both ```matplotlib``` and ```bqplot``` are used for plotting, with the latter used for the interactive plot.  For calculating Bernstein polynomials and polynomial functions, ```scipy``` has this built-in:  ```scipy.interpolate.BPoly```.  Since no other interactive widgets were used, ```ipywidgets``` was not imported.